In [42]:
from opensearchpy import OpenSearch
from elasticsearch.helpers import bulk
import json
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
import pandas as pd
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [52]:
host = 'localhost'
port = 9200
auth = ('admin', 'Joby.john@stacknexus42') 

In [54]:
{
  "settings": {
    "number_of_shards": 1,
    "number_of_replicas": 0
  },
  "mappings": {
    "properties": {
      "title": {
        "type": "text"
      },
      "owner": {
        "type": "text"
      },
      "tier": {
        "type": "keyword"
      },
      "version": {
        "type": "keyword"
      },
      "version_date": {
        "type": "date"
      },
      "description": {
        "type": "text"
      },
      "tags": {
        "type": "keyword"
      },
      "Health": {
        "properties": {
          "quality_score": {
            "type": "text"
          },
          "operational_health": {
            "type": "text"
          }
        }
      },
      "governance": {
        "properties": {
          "access_policies": {
            "type": "integer"
          },
          "data_policies": {
            "type": "integer"
          }
        }
      },
      "stacks": {
        "type": "keyword"
      },
      "document": {
        "type": "text"
      },
      "data_sources": {
        "type": "nested",
        "properties": {
          "title": {
            "type": "text"
          },
          "description": {
            "type": "text"
          }
        }
      }
    }
  }
}

{'settings': {'number_of_shards': 1, 'number_of_replicas': 0},
 'mappings': {'properties': {'title': {'type': 'text'},
   'owner': {'type': 'text'},
   'tier': {'type': 'keyword'},
   'version': {'type': 'keyword'},
   'version_date': {'type': 'date'},
   'description': {'type': 'text'},
   'tags': {'type': 'keyword'},
   'Health': {'properties': {'quality_score': {'type': 'text'},
     'operational_health': {'type': 'text'}}},
   'governance': {'properties': {'access_policies': {'type': 'integer'},
     'data_policies': {'type': 'integer'}}},
   'stacks': {'type': 'keyword'},
   'document': {'type': 'text'},
   'data_sources': {'type': 'nested',
    'properties': {'title': {'type': 'text'},
     'description': {'type': 'text'}}}}}}

In [58]:
import requests

# Constants
OPENSEARCH_HOST = 'localhost'
OPENSEARCH_PORT = '9200'
INDEX_NAME = "retail_dp_sample"
JSON_DATA_FILE = "Retail_DP_Sample.json"

# Prepare payload
with open(JSON_DATA_FILE, "r") as json_file:
    json_data = json.load(json_file)

payload = {}
payload["index"] = INDEX_NAME
payload["body"] = json_data

# Send POST request to OpenSearch
headers = {'Content-Type': 'application/json'}
response = requests.post(f"http://{OPENSEARCH_HOST}:{OPENSEARCH_PORT}/_bulk", headers=headers, data=json.dumps(payload))

# Print response
print(response.json())

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [31]:
# host = 'localhost'
# port = 9200
# ca_certs_path = '/path/to/trusted/CA/certificate.pem'

# client = OpenSearch(
#     hosts=[{'host': host, 'port': port}],
#     http_auth=('admin', 'admin'),
#     use_ssl=True,
#     verify_certs=True,
#     ca_certs=ca_certs_path
# )

In [44]:
with open("Retail_DP_Sample.json") as f:
    doc = json.load(f)

In [45]:
mapping = {
    "properties": {
        "title": {"type": "text"},
        "owner": {"type": "text"},
        "tier": {"type": "keyword"},
        "version": {"type": "keyword"},
        "version_date": {"type": "date"},
        "description": {"type": "text"},
        "document": {"type": "text"},
        "data_sources": {
            "properties": {
                "title": {"type": "text"},
                "description": {"type": "text"}
            }
        },
        "vector": {"type": "dense_vector", "dims": 768}  # Adjust dims as per your model
    }
}


In [48]:
client.indices.create('semantic_index', body={"mappings": mapping})

RequestError: RequestError(400, 'mapper_parsing_exception', 'No handler for type [dense_vector] declared on field [vector]')

In [ ]:
# Index the data with vector embeddings
for doc in json_data:
    text = " ".join([
        doc.get("description", ""),
        doc.get("document", ""),
        " ".join([source.get("description", "") for source in doc.get("data_sources", [])])
    ])

    # Use the plugin's vectorization capabilities
    vector = client.semantic_search.vectorize_text(text)

    doc_body = {
        "title": doc.get("title"),
        "owner": doc.get("owner"),
        "tier": doc.get("tier"),
        "version": doc.get("version"),
        "version_date": doc.get("version_date"),
        "description": doc.get("description", ""),
        "document": doc.get("document", ""),
        "data_sources": doc.get("data_sources", []),
        "vector": vector
    }

    client.index(index='semantic_index', body=doc_body)

# Search using vector similarity
query_text = "What is the customer 360 tool used for?"
query_vector = client.semantic_search.vectorize_text(query_text)

search_query = {
    "script_score": {
        "query": {"match_all": {}},
        "script": {
            "source": "cosineSimilarity(params.query_vector, 'vector') + 1.0",
            "params": {"query_vector": query_vector}
        }
    }
}

response = client.search(index="semantic_index", body=search_query)
print(response)

In [2]:
es = Elasticsearch("https://localhost:9200", basic_auth=("elastic", "Data_is_Lyfe"), verify_certs=False)

# Indexing in Elasticsearch

# Retail DP

In [4]:
with open("C360_updated.json") as f:
    doc = json.load(f)

In [5]:
doc.keys()

dict_keys(['title', 'description', 'owners', 'version', 'version_date', 'tags', 'health', 'data_sources', 'lens'])

In [6]:
len(doc["title_vec"])

KeyError: 'title_vec'

In [37]:
mapping_out = {'properties': {'description': {'type': 'text',
    'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
   'description_vec': {'type': 'dense_vector', 'dims':384, "index": True,"similarity": "l2_norm"},
   'title': {'type': 'text',
    'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
   'title_vec': {'type': 'dense_vector', 'dims':384, "index": True,"similarity": "l2_norm"}}}

In [36]:
es.indices.delete(index = "dp_test_1")

ObjectApiResponse({'acknowledged': True})

In [38]:
es.indices.create(index = "dp_test_1", mappings=mapping_out)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'dp_test_1'})

In [40]:
es.index(index = "dp_test_1", document=doc)

ObjectApiResponse({'_index': 'dp_test_1', '_id': 'vyBrhIsB_BrIaB8k4kGX', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

# Finance DP

In [41]:
with open("combined_finance.json") as f:
    fin_doc = json.load(f)

In [42]:
fin_out = {}
fin_out["title"] = fin_doc["title"]
fin_out["description"] = fin_doc["description"]
fin_out["title_vec"] = fin_doc["title_vec"]
fin_out["description_vec"] = fin_doc["description_vec"]

In [43]:
es.index(index = "dp_test_1", document=fin_out)

ObjectApiResponse({'_index': 'dp_test_1', '_id': 'wCBrhIsB_BrIaB8k60E9', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1})

# Healthcare DP

In [44]:
with open("combined_health.json") as f:
    health_doc = json.load(f)

In [45]:
health_out = {}
health_out["title"] = health_doc["title"]
health_out["description"] = health_doc["description"]
health_out["title_vec"] = health_doc["title_vec"]
health_out["description_vec"] = health_doc["description_vec"]

In [46]:
es.index(index = "dp_test_1", document=health_out)

ObjectApiResponse({'_index': 'dp_test_1', '_id': 'wSBrhIsB_BrIaB8k9EFd', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1})

# Semantic Search

In [152]:
query_phrase = "medicines"

In [153]:
qr_vec = model.encode(query_phrase)

In [154]:
qr = {
"script_score": {
# "min_score": 4.0,
"query": {"match_all": {}},
"script": {
"source": "2*(cosineSimilarity(params.query_vector, 'title_vec') + 1.0) + (cosineSimilarity(params.query_vector, 'description_vec') + 1.0)",
"params": {"query_vector": qr_vec}
}
}
}

In [112]:
qr_fuzz = {
"script_score": {
"query": {
    "fuzzy": {
      "title.keyword": {
        "value": "custumer"
      }
    }
},
"script": {
"source": "2*(cosineSimilarity(params.query_vector, 'title_vec') + 1.0) + (cosineSimilarity(params.query_vector, 'description_vec') + 1.0)",
"params": {"query_vector": qr_vec}
}
}
}

In [155]:
search_res = [{"title":t["_source"]["title"], "score":t["_score"]} for t in es.search(query=qr, index = "dp_test_1")["hits"]["hits"]]

In [156]:
search_res

[{'title': 'Healthcare_360', 'score': 3.716805},
 {'title': 'Customer_360', 'score': 3.2733693},
 {'title': 'Banking_360', 'score': 3.116454}]

In [52]:
search_1 = es.search(index="dp_test_1", body={"query": qr})

In [53]:
test_phrase = "ecommerce"

In [54]:
query_vec = model.encode(test_phrase)

In [55]:
len(query_vec)

384

In [56]:
qr = {
"script_score": {
"query": {"match_all": {}},
"script": {
"source": "2*(cosineSimilarity(params.query_vector, 'title_vec') + 1.0) + (cosineSimilarity(params.query_vector, 'description_vec') + 1.0)",
"params": {"query_vector": query_vec}
}
}
}

In [57]:
word_res = [{"title":t["_source"]["title"], "score":t["_score"]} for t in es.search(query=qr, index = "dp_test_1")["hits"]["hits"]]

In [58]:
word_res

[{'title': 'Banking_360', 'score': 4.0541496},
 {'title': 'Customer_360', 'score': 4.0235376},
 {'title': 'Healthcare_360', 'score': 3.4357424}]

# KNN search

In [162]:
test_phrase = "blood test"

In [163]:
query_vec = model.encode(test_phrase)

In [164]:
qr ={
    "field": "title_vec",
    "query_vector": query_vec,
    "k": 3,
    "num_candidates": 10
  }

In [165]:
word_res = [{"title":t["_source"]["title"], "score":t["_score"]} for t in es.knn_search(knn = qr, index = "dp_test_1")["hits"]["hits"]]

In [166]:
word_res

[{'title': 'Healthcare_360', 'score': 0.38176438},
 {'title': 'Banking_360', 'score': 0.35068586},
 {'title': 'Customer_360', 'score': 0.34415022}]